# Tensorflow

When starting off with deep learning, one of the first questions to ask is, which framework to learn?

Common choices include TensorFlow, PyTorch, and Keras. All of these choices have their own pros and cons and have their own way of doing things.

> From [**The Anatomy of Deep Learning Frameworks**](https://medium.com/@gokul_uf/the-anatomy-of-deep-learning-frameworks-46e2a7af5e47#.3ywhrk1st)

> The core components of a deep learning framework we must consider are:

> + How **Tensor Objects** are defined. At the heart of the framework is the tensor object. A tensor is a generalization of a matrix to n-dimensions. We need a Tensor Object that supports storing the data in form of tensors. Not just that, we would like the object to be able to convert other data types (images, text, video) into tensors and back, supporting indexing, overloading operators, having a space efficient way to store the data and so on.
+ How **Operations** on the Tensor Object are defined. A neural network can be considered as a series of Operations performed on an input tensor to give an output. 
+ The use of a **Computation Graph and its Optimizations**. Instead of implementing operations as functions, they are usually implemented as **classes**. This allows us to store more information about the operation like calculated shape of the output (useful for sanity checks), how to compute the gradient or the gradient itself (for the auto-differentiation), have ways to be able to decide whether to compute the op on GPU or CPU and so on. The power of neural networks lies in the ability to chain multiple operations to form a powerful approximator. Therefore, the standard use case is that you can initialize a tensor, perform actions after actions on them and finally interpret the resulting tensor as labels or real values. Unfortunately, as you chain more and more operations together, several issues arise that can drastically slow down your code and introduce bugs as well. There are more such issues and it becomes necessary to be able to get a bigger picture to even notice that these issues exist. We need a way to optimize the resultant chain of operations for both space and time. A Computation Graph which is basically an object that contains links to the instances of various Ops and the relations between which operation takes the output of which operation as well as additional information. 
+ The use of **Auto-differentiation** tools. Another benefit of having the computational graph is that calculating gradients used in the learning phase becomes modular and straightforward to compute. 
+ The use of **BLAS/cuBLAS and cuDNN** extensions for maximizing performance. BLAS or Basic Linear Algebra Subprograms are a collection of optimized matrix operations, initially written in Fortran. These can be leveraged to do very fast matrix (tensor) operations and can provide significant speedups. There are many other software packages like Intel MKL, ATLAS which also perform similar functions. BLAS packages are usually optimized assuming that the instructions will be run on a CPU. In the deep learning situation, this is not the case and BLAS may not be able to fully exploit the parallelism offered by GPUs. To solve this issue, NVIDIA has released cuBLAS which is optimized for GPUs. This is now included with the CUDA toolkit.

The computational model for Tensorflow (`tf`) is a **directed graph**.

**Nodes** are *functions* (*operations* in `tf` terminology) and **edges** are *tensors*. 

**Tensor** are multidimensional data arrays. 

$$f(a,b) = (a*b) + (a+b)$$

![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t1.png?raw=1)

There are several reasons for this design:
+ The most important is that is a good way to split up computation into small, **easily differentiable** pieces. `tf` uses automatic differentiation to automatically compute the derivative of every node with respect any other node that can affect the first node's output.
+ The graph is also a convenient way for distributing computation across multiple CPUs, GPUs, etc.

The primary API of `tf` (written in C++) is accessed through Python.


## Fundamentals

Tensorflow approaches series of computations as a flow of data through a graph with nodes being computation units and edges being flow of Tensors (multidimensional arrays).

Tensorflow builds the computation graph before it starts execution, so the computations are scheduled only when it is absolutely necessary (lazy programming).

TensorFlow comes with a tool, TensorBoard, to visualize the computation graph.

`tf` computation graphs are described in code with `tf` API.

In [1]:
import tensorflow as tf
print(tf.__version__)

1.12.0


> Python `with` statement (context manager) is useful when you have two related operations which you’d like to execute as a pair, with a block of code in between. The classic example is opening a file, manipulating the file, then closing it:

>```python
with open('output.txt', 'w') as f:
    f.write('Hi!')
>```

> The above `with` statement will automatically close the file after the nested block of code. The advantage of using a `with` statement is that it is guaranteed to close the file no matter how the nested block exits.

In [5]:
# Basic constant operations = to assign a value to a tensor

a = tf.constant(2)
b = tf.constant(3)
c = a+b 
d = a*b
e = c+d

# non interactive session
# the context manager will automatically close the session

with tf.Session() as sess:
    print("a= %i" % sess.run(a))
    print("b= %i" % sess.run(b))
    print("(a+b)+(a*b) = %i" % sess.run(e))

a= 2
b= 3
(a+b)+(a*b) = 11


`sess.run(node)` executes the part of the computational graph that is needed to compute the value of `node` and only that part. While defining the graph, we are not manipulating any data, only building the nodes and symbols inside our graph.

We can use `tf.get_default_graph().get_operations()` to see all the nodes in the graph.

In [3]:
tf.get_default_graph().get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'add_1' type=Add>]

You can create initialized tensors in many ways:

In [4]:
a = tf.zeros([2,3], tf.int32)
b = tf.ones([2,3], tf.int32)
c = tf.fill([3,3], 23.9)
d = tf.range(0,10,1)

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))
    print(sess.run(d))

[[0 0 0]
 [0 0 0]]
[[1 1 1]
 [1 1 1]]
[[23.9 23.9 23.9]
 [23.9 23.9 23.9]
 [23.9 23.9 23.9]]
[0 1 2 3 4 5 6 7 8 9]


``tf`` sequences are not iterable!

We can also generate random variables:

In [5]:
a = tf.random_normal([2,2], 0.0, 1.0)
b = tf.random_uniform([2,2], 0.0, 1.0)

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))

[[ 0.83202136 -1.335937  ]
 [ 1.834337   -0.33183086]]
[[0.3809768  0.12681663]
 [0.5305053  0.40903056]]


How to generate random shuffled number in tensorflow?

In [6]:
idx = tf.constant(20)
idx_list = tf.range(idx) # 0~19
shuffle = tf.random_shuffle(idx_list)

# in this case tf returns, in a list, two diferent results
with tf.Session() as sess:
    a, b = sess.run([idx_list, shuffle])
    print(a)
    print(b)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 5  8 10 15  9 12 18  2  6 11 13  4  3 19  0 14 17  1  7 16]


Essentially, TensorFlow computation graph contains the following parts:

+ **Placeholders**, variables used in place of inputs to feed to the graph
+ **Variables**, model variables that are going to be optimized to make model perform better
+ **Model**, a mathematical function that calculates output based on placeholder and model variables

When used as an optimization engine, we will also have:
+ **Loss Measure**, guide for optimization of model variables
+ **Optimization Method**, update method for tuning model variables

In [7]:
# Basic operations with variable graph input

a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)
c = tf.add(a,b) 
d = tf.multiply(a,b)
e = tf.add(c,d)

values = feed_dict={a: 5, b: 3}

# non interactive session

with tf.Session() as sess:
    print('a = %i' % sess.run(a, values))
    print('b = %i' % sess.run(b, values))
    print("(a+b)+(a*b) = %i" % sess.run(e, values))

a = 5
b = 3
(a+b)+(a*b) = 23


A computational graph is a series of functions chained together, each passing its output to zero, one or more functions further along the chain.

In this way we can construct very complex transformations on data by using a library of simple functions.

Nodes represent some sort of computation being done in the graph context.

Edges are the actual values (tensors) that get passed to and from nodes.

![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t2.png?raw=1)

+ The values flowing into the graph can come from different sources: from a different graph, from a file, entered by the client, etc. The *input* nodes simply pass on values given to them.
+ The other nodes take values, apply an operation and output their result. 

Values running on edges are tensors:

In [8]:
# Basic operations with variable as graph input

a = tf.placeholder(tf.int16,shape=[2])
b = tf.placeholder(tf.int16,shape=[2])
c = tf.add(a,b) 
d = tf.multiply(a,b)
e = tf.add(c,d)

variables = feed_dict={a: [2,2], b: [3,3]}

# non interactive session

with tf.Session() as sess:
    print(sess.run(a, variables))
    print(sess.run(b, variables))
    print(sess.run(e, variables))

[2 2]
[3 3]
[11 11]


### Exercise

Implement this computational graph:

![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t3.png?raw=1)


In [9]:
# your code here


There are certein connections between nodes that are not allowed: you cannot create **circular dependencies**.

> Dependency: Any node `A` that is required for the computation of a later node `B` is said to be a **dependency** of `B`.

The main reason is that dependencies create endless feedback loops.

There is one exception to this rule: *recurrent neural networks*. In this case `tf` simulate this kind of dependences by copying a **finite** number of versions of the graph, placing them side-by-side, and feeding them into another sequence. This process is referred as **unrolling** the graph.

![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t4.png?raw=1)

Keeping track of dependencies is a basic feature of `tf`. Let's suppose that we want to compute the output value of the `mul` node. We can see in the unrolled graph that is not necessary to compute the full graph to get the output of that node. But how to ensure that we only compute the necessary nodes?

It's pretty easy:
+ Build a list for each node with all nodes it directly depends on (not indirectly!). 
+ Initialize an empty stack, wich will eventually hold all the nodes we want to compute. 
+ Put the node you want to get the output from. 
+ Recursively, look at its dependency list and add to the stack the nodes it depends on, until there are no dependencies left to run and in this way we guarantee that we have all the nodes we need.

The stack will be ordered in a way that we are guaranteed to be able to run each node in the stack as we iterate through it. 

The main thing to look out for is to keep track of nodes that were already computed and to store their value in memory.


## `tf` workflows

As we have seen in previous code, `tf` workflow is a two-step process:

+ Define the computation graph.
+ Run the graph with data.

This can be done in a *non interactive* mode or in an *interactive* mode.

In [10]:
# new graph definition
tf.reset_default_graph()

# we can assign a name to every node
a = tf.placeholder(tf.int32, name='input_a')
b = tf.placeholder(tf.int32, name='input_b')
c = tf.add(a,b,name='add_1') 
d = tf.multiply(a,b,name='mul_1')
e = tf.add(c,d,name='add_2')

values = feed_dict={a: 5, b: 3}

# now we can run the graph in an interactive session
sess = tf.Session()
print(sess.run(e, values))

# it is our responsability to close the session
sess.close()

tf.get_default_graph().get_operations()

23


[<tf.Operation 'input_a' type=Placeholder>,
 <tf.Operation 'input_b' type=Placeholder>,
 <tf.Operation 'add_1' type=Add>,
 <tf.Operation 'mul_1' type=Mul>,
 <tf.Operation 'add_2' type=Add>]

`tf` has a very useful tool: `tensor-board`. Let's see how to use it. 

In [7]:
# cleaning the tf graph space
tf.reset_default_graph()

a = tf.placeholder(tf.int16, name='input_a')
b = tf.placeholder(tf.int16, name='input_b')
c = tf.add(a,b,name='add_1') 
d = tf.multiply(a,b,name='mul_1')
e = tf.add(c,d,name='add_2')

values = feed_dict={a: 5, b: 3}

# now we can run the graph

# graphs are run by invoking Session objects
session = tf.Session()

# when you are passing an operation to 'run' you are 
# asking to run all operations necessary to compute that node

# you can save the value of the node in a Python var
output = session.run(e, values)
print(output)

# now let's visualize the graph

# SummaryWriter is an object where we can save information
# about the execution of the computational graph
writer = tf.summary.FileWriter('my_graph', session.graph)
writer.close()

# closing interactive session
session.close()


23


Open a terminal, go to your working dir, and type in:

`tensorboard --logdir="my_graph"`

This starts a `tensorboard` server on port 6006. There, click on the `Graphs` link. You can see that each of the nodes is labeled based on the `name` parameter you passed into each operation.

### Exercise

Implement and visualize this graph for a constant tensor `[5,3]`:

![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t5.png?raw=1)

Check these functions in the `tf` official documentation (https://www.tensorflow.org/): `tf.math.reduce_prod`, `tf.math.reduce_sum`.


In [ ]:
# your code here


## `tf` statements

### `tf` input data 

`tf` can take several Python var types that are automatically converted to tensors:

`tf.constant([5,3], name='input_a')`

But `tf` has a plethora of other data types: `tf_int16`, `tf_quint8`, etc.

`tf` is tightly integrated with NumPy. In fact, `tf` data types are based on those from NumPy. Tensors returned from `Session.run` are NumPy arrays. NumPy arrays is the recommended way of specifying tensors.

The `shape` of tensors describe both the number of dimensions in a tensor  as well as the length of each dimension. In addition to to being able to specify fixed lengths to each dimension, in some situations you can also assign a flexible length by passing in `None` as dimension's value.


In [9]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

a = tf.placeholder(tf.int16, shape=[2,2], name='input_a')
shape = tf.shape(a)

session = tf.Session()
print(session.run(shape))
session.close()

[2 2]


We can feed data points to placeholder by iterating through the data set:

In [13]:
tf.reset_default_graph()

list_a_values = [1,2,3]

a = tf.placeholder(tf.int16)
b = a * 2

with tf.Session() as sess:
    for a_value in list_a_values:
        print(sess.run(b,{a: a_value}))

2
4
6


### `tf` operations

`tf` overloads common mathematical operations:

In [10]:
import tensorflow as tf

tf.reset_default_graph()

a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)
c = a+b 
d = a*b
e = tf.math.add(c,d) #equivalent to c+d

variables = feed_dict={a: 5, b: 3}

with tf.Session() as sess:
    print("(a+b)+(a*b) = %i" % sess.run(e, variables))

(a+b)+(a*b) = 23


There are more [Tensorflow Operations](https://www.tensorflow.org/api_guides/python/math_ops)

### `tf` graphs

Creating a graph is simple:

```python
import tensorflow as tf
g = tf.Graph()
```

Once the graph is initialized we can attach operation to it by using the `Graph.as_default()` method:

```python
with g.as_default():
    a = tf.mul(2,3)
    ...
```

`tf` automatically creates a graph at the beginning and assigns it to be the default. Thus, if not using `Graph.as_default()` any operation will be automatically placed in the default graph.

Creating multiple graphs can be useful if you are defining multiple models that do not have interdependencies:

```python
g1 = tf.Graph()
g2 = tf.Graph()

with g1.as_default():
    ...
    
with g2.as_default():
    ...
```

### `tf` Variables

`Tensor` and `Operation` objects are **immutable**, but we need a mechanism to save changing values over time (persisting during several `run`). 

This is accomplished with `Variable` objects, which contain mutable tensor values that persist accross multiple calls to `Session.run()`.

Variables can be used anywhere you might use a tensor.

`tf` has a number of helper operations to initialize variables: `tf-zeros()`, `tf_ones()`, `tf.random_uniform()`, `tf.random_normal()`, etc.

`Variable` objects live in a `Graph` but their state is managed by `Session`. 

Because of these they need an extra step for inicialization (`tf.global_variables_initializer`):

```python
import tensorflow as tf

a = tf.Variable(3,name="my_var")
b = tf.add(5,a)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ...

```

In order to chage the value of a `Variable` we can use the `Variable.assign()` method:

In [22]:
import tensorflow as tf

a = tf.Variable(3,name="my_var")
b = a.assign(tf.multiply(2,a))  

# The statement a.assign(...) does not actually assign any to a, 
# but rather creates a tf.Operation that you have to explicitly 
# run to update the variable.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("a:", a.eval())   # variables are objects, not ops.
    print("b:", sess.run(b))
    print("b:", sess.run(b))
    print("b:", sess.run(b))
    print("a:", a.eval())

a: 3
b: 6
b: 12
b: 24
a: 24


In [23]:
tf.reset_default_graph()

a = tf.Variable(3,name="my_var")
b = a.assign(tf.multiply(2,a))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(a.eval())
    print(b.eval())

3
6


We can increment and decrement variables: 

In [20]:
import tensorflow as tf

a = tf.Variable(3,name="my_var")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a.assign_add(1)))
    print(sess.run(a.assign_sub(1)))
    print(sess.run(a.assign_sub(1)))
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))

4
3
2
3


Some classes of `tf` (f.e. `Optimizer`) are able to automatically change variable values without explicitely asking to do so.

Tensorflow sessions maintain values separately, each `Session` can have its own current value for a variable defined in the graph:

In [19]:
tf.global_variables_initializer()

a = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(tf.global_variables_initializer())
sess2.run(tf.global_variables_initializer())

print(sess1.run(a.assign_add(10)))
print(sess2.run(a.assign_sub(2)))

sess1.close()
sess2.close()

20
8


### `tf` name scopes

`tf` offers a tool to help organize your graphs: name scopes.

Name scopes allows you to group operations into larger, named blocks. This is very usefu to visualize complex models with `tensorboard`. 

In [20]:
import tensorflow as tf
tf.reset_default_graph()

with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name="A_add")
    b = tf.multiply(a, 3, name="A_mul")

with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = tf.multiply(c, 6, name="B_mul")

e = tf.add(b, d, name="output")

writer = tf.summary.FileWriter('./name_scope_1', graph=tf.get_default_graph())
writer.close()

with tf.Session() as sess:
    print(sess.run(e))

sess.close()

63


We can start `tensorboard` to see the graph: `tensorboard --logdir="./name_scope_1"`.

You can expand the name scope boxes by clicking `+`. 

### Exercise

Let's built and visualize and complex model:

+ Our inputs will be placeholders.
+ The model will take in a single vector of any lenght.
+ The graph will be segmented in name scopes.
+ We will accumulate the total value of all outputs over time.
+ At each run, we are going to save the output of the graph, the accumulated total of all outputs, and the average value of all outputs to disk for use in `tensorboard`.


![alt text](https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/t6.png?raw=1)


In [ ]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
    # your code here

    
    # Primary transformation Operations
    with tf.name_scope("transformation"):
        
        # Separate input layer
        with tf.name_scope("input"):
        # your code here

    
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
        # your code here

        
        # Separate output layer
        # your code here
        
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        # your code here

    
    # Summary Operations
    with tf.name_scope("summaries"):
        # Calculating average (avg = total/steps)
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")

        # Creates summaries for output node
        tf.summary.scalar("output_summary", output)
        tf.summary.scalar("total_summary", update_total)
        tf.summary.scalar("average_summary", avg)
    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()
        # Merge all summaries[…]
        merged_summaries = tf.summary.merge_all()

# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('./improved_graph', graph)

# Initialize Variables
sess.run(init)

Let's write a function to run the graph several times:

In [ ]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}
    out, step, summary = sess.run([output, increment_step, merged_summaries], 
                                  feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [ ]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

# Write the summaries to disk
writer.flush()

# Close the SummaryWriter
writer.close()

# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

`tensorboard --logdir='./improved_graph'`   

## ``tf`` eager execution

``Eager`` execution is an interface to TensorFlow that provides an imperative programming style. When you enable eager execution, TensorFlow operations execute immediately; you do not execute a pre-constructed graph with ``Session.run()``.

First, we must enable ``Eager`` execution. When we do this, operations will execute and return their values immediately. Some things to note:

+ We will need to restart the Python kernel since we have already used TensorFlow in graph mode.
+ We enable eager at program startup using: tfe.enable_eager_execution().
+ Once we enable Eager with ``tfe.enable_eager_execution()``, it cannot be turned off. To get back to graph mode, start a new Python session.

In [1]:
import tensorflow.contrib.eager as tfe
import tensorflow as tf

tfe.enable_eager_execution()


In [2]:
print(tf.add(1, 2))

tf.Tensor(3, shape=(), dtype=int32)


TensorFlow Eager execution provides an autograd style API for automatic differentiation.

In [3]:
def f(x):
    # f(x) = x^2 + 3
    return tf.multiply(x, x) + 3

print( "f(4) = %.2f" % f(4.) )

# First order derivative
df = tfe.gradients_function(f)
print( "df(4) = %.2f" % df(4.)[0] )

# Second order derivative
d2f = tfe.gradients_function(lambda x: df(x)[0])
print( "d2f(4) = %.2f" % d2f(4.)[0] )

f(4) = 19.00
df(4) = 8.00
d2f(4) = 2.00
